<a href="https://colab.research.google.com/github/shinjeongdong/MLDeeplearningStudy/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%94%A5%EB%9F%AC%EB%8B%9D_5%EC%9E%A5_%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np
import importlib
import sys
import pickle
import copy
importlib.invalidate_caches()
sys.path.append('/content/drive/MyDrive/dataset')
from func import softmax,cross_entropy_error

In [130]:
#역전파는 신경망의 각 노드가 가지고 있는 가중치(Weight)와 편향(Bias)을 학습시키기 위한 알고리즘으로, 딥러닝에 있어서 가장 핵심적인 부분이라고 할 수 있다.
#목표(Target)와 모델의 예측 결과(Output)가 얼마나 차이가 나는지 확인하고 그 오차를 바탕으로 가중치와 편향을 뒤에서부터 앞으로 갱신해가는 것을 의미한다.
#역전파란 명칭도 바로 이처럼 뒤에서부터 다시 앞으로 거슬러 올라간다는 것에서 나온 것이다.
#핵심은 chain rule 합성함수의 미분법을 활용하는것이다.
#ex 손실함수 L이랑 활성함수 y_i = f(w_i)가 있을때 L(f(w_i))일때∂L/∂w_i을 구하고 싶을땐
#∂L/∂y_i * ∂y_i/∂w_i을 하면 분수형태로 만들어져 ∂y_i ∂y_i는 지워져 ∂L/∂w_i만 남게 된다.
#말로 하면 어려우니 역전파 관련 유튜브를 찾아보는걸 추천
#∂

In [131]:
#활성함수 계층 구현하기
#Relu y = x (x>0) else 0
#∂y/∂x if x>0,1 else, 0

class Relu:
  def __init__(self):
    self.mask = None

  def forward(self,x):
    self.mask = (x<=0)
    out = x.copy()

    out[self.mask] = 0
    return out

  def backward(self,dout):
    dout[self.mask] = 0
    dx = dout

    return dx

In [132]:
#sigmoid
#1 / 1+exp(-x) 역전파 = ∂L/∂y * sigmoid미분

class sigmoid:
  def __init__(self):
    self.out = None

  def forward(self,x):
    out = 1 / (1+np.exp(-x))
    self.out = out
    return out

  def backward(self,dout):

    dx = dout * (1.0 - self.out) * self.out

    return dx

In [133]:
#배치용 Affine 계층


class Affine:
  def __init__(self,W,b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
  def forward(self,x):#순전파
    self.x = x
    out = np.dot(x,self.W) + self.b  #x가 들어오면 W랑 행렬곱 + b을해준다

    return out

  def backward(self,dout): #역전파
    dx = np.dot(dout, self.W.T) #∂L/∂x 구하는법 미분값 * W의 T를 곱해준다.

    self.dW = np.dot(self.x.T,dout) #W의 기울기 벡터는 x.T dout를 곱해준다.
    self.db = np.sum(dout,axis = 0)

    return dx


In [134]:
#softmax + loss

class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y =  None # softmax의 출력 = 예측값
    self.t = None #정답 데이터 원핫 인코딩

  def forward(self,x,t): #소프트맥스와 손실함수
    self.t = t #정답데이터
    self.y = softmax(x) #Affine2 순전파가 완료된 값을 softmax로 각각의 확률을 구해준다.
    self.loss = cross_entropy_error(self.y,self.t) #그리고 예측값과 정답값을 넣어줘 손실을 구해준다.
    return self.loss

  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size #Softmax랑 로스함수의 역전파는 (y_i - t_i)이다 간단

    return dx


In [135]:
#역전파를 이용한 신경망 구현 알고리즘
#1단계 미니배치 훈련 데이터중 일부를 무작위로 가져옴 이 미니배치의 손실함수 값을 줄이는게 목표
#2단계 기울기 산출 미니배치의 손실함수를 줄이기 위해 기울기벡터를 구해준다
#3단계 가중치를 기울기방향으로 조금 갱신한다
#4단계 1~3단계 반복
#일단 2층 신경망을 구현해보자
from func import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
    self.params['b2'] = np.zeros(output_size)

    #계층생성
    self.layers = OrderedDict() #딕셔너리 생성
    self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1']) #클래스를 생성해줘서 W1 b1 인스턴스 변수들을 할당한다.
    self.layers['Relu1'] = Relu() #클래스를 생성해줘서 인스턴스 변수들을 할당한다.
    self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2']) #클래스를 생성해줘서 W2 b2 인스턴스 변수들을 할당한다.

    self.lastlayer = SoftmaxWithLoss() #이 Affine2까지 진행된 순전파를 이용해 예측 확률값과 손실값 인스턴스 변수를 만든다.

  def predict(self,x):
    for layer in self.layers.values(): #순전파 부분.

      x = layer.forward(x) #affine 1 relu1 affine2 순서로 순전파를 함수를 실행해준다.

    return x

  def loss(self,x,t):
    y = self.predict(x) #위의 layers의 순전파를 실행

    return self.lastlayer.forward(y,t) # softmaxwithloss 실행

  def accuracy(self,x,t):
    y = self.predict(x)
    y = np.argmax(y,axis=1)
    if t.ndim != 1 : t = np.argmax(t,axis = 1)

    accuracy = np.sum(y == t) / float(x.shape[0])

    return accuracy

  def numerical_gradient(self,x,t): #전 쳅터에서 구현한 부분.
    loss_W = lambda W: self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads


  def gradient(self,x,t):
    #순전파
    self.loss(x,t) # loss를 실행하여 Affine1, relu, Affine2함수의 모든 인스턴스 변수를 갱신해준다.


    #역전파

    dout = 1

    dout = self.lastlayer.backward(dout) #softmaxwithloss의 역전파를 실행한다. (y-t)

    layers = list(self.layers.values())
    layers.reverse() #역전파는 순전파의 반대방향으로 실행함 Affine2 -> Relu1 -> Affine1 순으로 그래서 역전환 함.

    for layer in layers:
      dout = layer.backward(dout) #반대로 만든것을 실행.

    grads = {}
    grads['W1'] = self.layers['Affine1'].dW #Affine1또는2 에 있는 인스턴스 변수인 W,b의 기울기 이하 동문.
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db

    return grads

In [136]:
from mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True) #각각의 데이터는 (60000,784), (60000,100), (10000,784), (10000,100) 형식

network = TwoLayerNet(784,50,10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size,1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size,batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  grad = network.gradient(x_batch,t_batch)

  for key in ('W1','b1','W2','b2'):
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch,t_batch)
  train_loss_list.append(loss)

  if i% iter_per_epoch ==0:
    train_acc = network.accuracy(x_train,t_train)
    test_acc = network.accuracy(x_test,t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc,test_acc) # 0.09843333333333333 0.1031 -> 0.9781333333333333 0.9702

0.09843333333333333 0.1031
0.9037666666666667 0.9097
0.9239833333333334 0.9267
0.93425 0.935
0.94245 0.9389
0.9480166666666666 0.9456
0.9550333333333333 0.952
0.9589333333333333 0.9562
0.9628666666666666 0.9584
0.96555 0.9604
0.9668833333333333 0.9625
0.96895 0.9632
0.9725166666666667 0.9653
0.9735 0.9659
0.9758666666666667 0.9679
0.9767166666666667 0.9684
0.9781333333333333 0.9702


In [137]:
#간단한 테스트
net = TwoLayerNet(2,3,2)

net.params['W1'] = np.array([[0.1,0.2,0.3],[0.6,0.5,0.4]])

net.params['W2'] = np.array([[0.3,0.1],[0.2,0.5],[0.8,0.7]])


x = np.array([[2,3]])
t = np.array([[0,1]])

net.layers = OrderedDict() #딕셔너리 생성
net.layers['Affine1'] = Affine(net.params['W1'],net.params['b1']) #클래스를 생성해줘서 W1 b1 인스턴스 변수들을 할당한다.
net.layers['Relu1'] = Relu() #클래스를 생성해줘서 인스턴스 변수들을 할당한다.
net.layers['Affine2'] = Affine(net.params['W2'],net.params['b2']) #클래스를 생성해줘서 W2 b2 인스턴스 변수들을 할당한다.

net.loss(x,t) #0.698159479502866

gd = net.gradient(x,t)

net.params['W1'] -= 1 * gd['W1']

net.loss(x,t)


for i in range(0,9):
  net.params['W1'] -= 1 * gd['W1']



net.loss(x,t)



np.float64(0.0015803586491286087)